# Imports

In [167]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import sqlite3

# Read in MIPS data

In [168]:
mips = pd.read_csv('data/ec_score_file.csv')

In [169]:
mips.columns = [col.replace(" ", "").lower() for col in mips.columns]

In [170]:
mips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 933504 entries, 0 to 933503
Data columns (total 13 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   npi                           933504 non-null  int64  
 1   org_pac_id                    930316 non-null  float64
 2   lst_nm                        933469 non-null  object 
 3   frst_nm                       933488 non-null  object 
 4   source                        933504 non-null  object 
 5   facility_ccn                  23904 non-null   float64
 6   facility_lbn                  23901 non-null   object 
 7   quality_category_score        853226 non-null  float64
 8   pi_category_score             595380 non-null  float64
 9   ia_category_score             865483 non-null  float64
 10  cost_category_score           0 non-null       float64
 11  final_mips_score_without_cpb  933504 non-null  float64
 12  final_mips_score              933504 non-nul

In [171]:
mips.head()

,npi,org_pac_id,lst_nm,frst_nm,source,facility_ccn,facility_lbn,quality_category_score,pi_category_score,ia_category_score,cost_category_score,final_mips_score_without_cpb,final_mips_score
0,1003028101,NaN,KATZ,KENNETH,individual,NaN,NaN,NaN,NaN,NaN,NaN,45.0,45.0
1,1003028556,NaN,HEIDARI,NEDA,individual,NaN,NaN,NaN,NaN,NaN,NaN,45.0,45.0
2,1003156209,NaN,GUERRA DEL CASTILLO,ROBERTO,apm,NaN,NaN,NaN,NaN,NaN,NaN,45.0,45.0
3,1003162173,NaN,HASSAN KAMEL,MOHAMED,apm,NaN,NaN,NaN,NaN,NaN,NaN,45.0,45.0
4,1003220351,NaN,NAUMOVA,YULIA,individual,NaN,NaN,NaN,NaN,NaN,NaN,45.0,45.0


In [172]:
mips_cols = ['npi', 
             'quality_category_score', 
             'pi_category_score', 
             'ia_category_score', 
             'final_mips_score_without_cpb', 
             'final_mips_score']

In [173]:
mips_thin = mips[mips_cols]

In [174]:
mips_thin

,npi,quality_category_score,pi_category_score,ia_category_score,final_mips_score_without_cpb,final_mips_score
0,1003028101,NaN,NaN,NaN,45.0,45.0
1,1003028556,NaN,NaN,NaN,45.0,45.0
2,1003156209,NaN,NaN,NaN,45.0,45.0
3,1003162173,NaN,NaN,NaN,45.0,45.0
4,1003220351,NaN,NaN,NaN,45.0,45.0
...,...,...,...,...,...,...
933499,1265429369,90.4,92.0,40.0,92.3,98.1
933500,1649554882,94.4,89.0,40.0,93.6,98.1
933501,1811984917,89.8,95.0,40.0,92.9,96.2
933502,1891957023,85.3,90.0,40.0,88.9,93.6


# Read in Sample Sets

In [175]:
tncosts_full = pd.read_csv('data/Medicare_Physician_Other_Practitioners_by_Provider_and_Service_2019_TN.csv')
tncosts_full.columns

Index(['Rndrng_NPI', 'Rndrng_Prvdr_Last_Org_Name', 'Rndrng_Prvdr_First_Name',
       'Rndrng_Prvdr_MI', 'Rndrng_Prvdr_Crdntls', 'Rndrng_Prvdr_Gndr',
       'Rndrng_Prvdr_Ent_Cd', 'Rndrng_Prvdr_St1', 'Rndrng_Prvdr_St2',
       'Rndrng_Prvdr_City', 'Rndrng_Prvdr_State_Abrvtn',
       'Rndrng_Prvdr_State_FIPS', 'Rndrng_Prvdr_Zip5', 'Rndrng_Prvdr_RUCA',
       'Rndrng_Prvdr_RUCA_Desc', 'Rndrng_Prvdr_Cntry', 'Rndrng_Prvdr_Type',
       'Rndrng_Prvdr_Mdcr_Prtcptg_Ind', 'HCPCS_Cd', 'HCPCS_Desc',
       'HCPCS_Drug_Ind', 'Place_Of_Srvc', 'Tot_Benes', 'Tot_Srvcs',
       'Tot_Bene_Day_Srvcs', 'Avg_Sbmtd_Chrg', 'Avg_Mdcr_Alowd_Amt',
       'Avg_Mdcr_Pymt_Amt', 'Avg_Mdcr_Stdzd_Amt'],
      dtype='object')

In [176]:
tncosts_cols = ['Rndrng_NPI', 
                'Rndrng_Prvdr_Last_Org_Name', 
                'Rndrng_Prvdr_First_Name',
                'Rndrng_Prvdr_Crdntls',
                'Rndrng_Prvdr_Type', 
                'HCPCS_Cd', 
                'HCPCS_Desc',
                'Place_Of_Srvc',
                'Tot_Benes', 
                'Tot_Srvcs',
                'Tot_Bene_Day_Srvcs',
                'Avg_Sbmtd_Chrg', 
                'Avg_Mdcr_Alowd_Amt', 
                'Avg_Mdcr_Pymt_Amt', 
                'Avg_Mdcr_Stdzd_Amt']

tncosts = tncosts_full[tncosts_cols]

In [177]:
tncosts.columns = [col.replace('Rndrng_', '').replace("Prvdr_", "").lower() for col in tncosts.columns]

In [178]:
rename_dict = {
    'type': 'specialty',
    'last_org_name':'last_name', 
    'first_name': 'first_name',
    'crdntls': 'credentials',
    'hcpcs_cd': 'procedure_code', 
    'hcpcs_desc': 'procedure_description',
    'place_of_srvc': 'place_of_service',
    'tot_benes': 'total_beneficiaries', 
    'tot_srvcs': 'total_services',
    'tot_bene_day_srvcs': 'total_beneficiaries_per_day_services',
    'avg_sbmtd_chrg': 'avg_submitted_charge', 
    'avg_mdcr_alowd_amt': 'avg_medicare_allowed', 
    'avg_mdcr_pymt_amt': 'avg_medicare_payment',
    'avg_mdcr_stdzd_amt': 'avg_medicare_standardized_payment'
}

tn_costs = tncosts.rename(columns = rename_dict)

In [179]:
tn_costs.columns

Index(['npi', 'last_name', 'first_name', 'credentials', 'specialty',
       'procedure_code', 'procedure_description', 'place_of_service',
       'total_beneficiaries', 'total_services',
       'total_beneficiaries_per_day_services', 'avg_submitted_charge',
       'avg_medicare_allowed', 'avg_medicare_payment',
       'avg_medicare_standardized_payment'],
      dtype='object')

# Modify the tom_cruise DB

In [180]:
with sqlite3.connect('data/tom_cruise.sqlite') as db: 
    cursor = db.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    print(cursor.fetchall())
    cursor.close()

[('profile',), ('taxonomy',), ('zip_cbsa',), ('affiliations',)]


In [181]:
# tables = ['referral_from', 'referral_to', 'referrals']

# with sqlite3.connect('data/tom_cruise.sqlite') as db:
    
#     for table in tables:
    
#         query = f"""

#         DROP TABLE {table};

#         """
        
#         db.execute(query)

# Read in geocodes

In [182]:
geo = pd.read_csv('data/to_address_geocodes_big.csv').rename(columns = {"address_to":"address"})
geo

,address,lat,lng
0,"400 E PUBLIC SQUARE, ALEXANDRIA, TN 37012",36.077559,-86.033322
1,"5801 CROSSINGS BLVD, ANTIOCH, TN 37013",36.037696,-86.642255
2,"5323 MOUNT VIEW RD, ANTIOCH, TN 37013",36.048716,-86.647766
3,"889B BELL RD, STE A-7A, ANTIOCH, TN 37013",36.050291,-86.650860
4,"777 BELL RD, ANTIOCH, TN 37013",36.051049,-86.649262
...,...,...,...
3813,"112 BRADFORD BLVD STE 500, GORDONSVILLE, TN 38563",36.178176,-85.949145
3814,"126 JMZ DR, GORDONSVILLE, TN 38563",36.190714,-85.951012
3815,"112 BRADFORD BLVD, SUITE 500, GORDONSVILLE, TN...",36.178176,-85.949145
3816,"8 NEW MIDDLETON HWY, STE A, GORDONSVILLE, TN 3...",36.173668,-85.947819


# Pull profiles and taxonomies from tom_cruise db

In [183]:
%%time
# referrals from providers (entity 1) to metro nashville groups (entity 2) 
# having over 50 transactions and under 50 average wait time
query = """

    SELECT
            npi,
            entity_type_code,
            "provider_last_name_(legal_name)" || ', ' || provider_first_name AS name,
            taxonomy_code,
            classification,
            specialization,
            "provider_organization_name_(legal_business_name)" AS organization,
            provider_first_line_business_practice_location_address AS address_line1,
            provider_second_line_business_practice_location_address AS address_line2,
            provider_business_practice_location_address_city_name AS city,
            provider_business_practice_location_address_state_name AS state,
            provider_business_practice_location_address_postal_code AS zip
    FROM profile
    INNER JOIN taxonomy
    ON profile.taxonomy_code = taxonomy.code
    WHERE provider_business_practice_location_address_postal_code IN (
        SELECT zip
        FROM zip_cbsa
        WHERE cbsa = '34980'
        )

"""
with sqlite3.connect('data/tom_cruise.sqlite') as db:
    profiles = pd.read_sql(query, db)

profiles

Wall time: 1.25 s


,npi,entity_type_code,name,taxonomy_code,classification,specialization,organization,address_line1,address_line2,city,state,zip
0,1346216199,1.0,"GERTONSON, STEVEN",367500000X,"Nurse Anesthetist, Certified Registered",None,None,1950 BRADBURY RD,None,ADAMS,TN,37010
1,1649660085,1.0,"SMITH, KALEENA",235Z00000X,Speech-Language Pathologist,None,None,948 NECTAR CT,None,ADAMS,TN,37010
2,1811367980,1.0,"GRAMLICK, RENE",133V00000X,"Dietitian, Registered",None,None,2875 OLD CLARKSVILLE SPGFLD RD,None,ADAMS,TN,37010
3,1760929236,1.0,"HUFF, SYDNEY",390200000X,Student in an Organized Health Care Education/...,None,None,3835 HEAD RD,None,ADAMS,TN,37010
4,1740720275,2.0,None,261QM1300X,Clinic/Center,Multi-Specialty,CHS TENNESSEE MEDICAL PC,4538 GUTHRIE HWY,None,CLARKSVILLE,TN,37010
...,...,...,...,...,...,...,...,...,...,...,...,...
44218,1194394767,1.0,"VADEN, CHARLI",225100000X,Physical Therapist,None,None,417 GORDONSVILLE HWY STE 100,None,GORDONSVILLE,TN,38563
44219,1518611425,1.0,"SIRCY, ANNA",363LF0000X,Nurse Practitioner,Family,None,8 NEW MIDDLETON HWY,None,GORDONSVILLE,TN,38563
44220,1831561059,1.0,"MONTALBO, CYNTHIA",224Z00000X,Occupational Therapy Assistant,None,None,73 JUDKINS LN,None,HICKMAN,TN,38567
44221,1558875468,2.0,None,343900000X,Non-emergency Medical Transport (VAN),None,"BLUE HERON TRANSPORTATION, LLC",177 SPURLOCK LN,None,HICKMAN,TN,38567


# Build full addresses

In [184]:
def build_address(df):
    address = ((df['address_line1'] + ', ' + df['address_line2']).mask(pd.isna, df['address_line1'])
         + ', ' 
         + df['city'] 
         + ', ' 
         + df['state']
         + ' '
         + df['zip']
    )
    return address

In [185]:
profiles['address'] = build_address(profiles)

,npi,entity_type_code,name,taxonomy_code,classification,specialization,organization,address_line1,address_line2,city,state,zip,address
0,1346216199,1.0,"GERTONSON, STEVEN",367500000X,"Nurse Anesthetist, Certified Registered",None,None,1950 BRADBURY RD,None,ADAMS,TN,37010,"1950 BRADBURY RD, ADAMS, TN 37010"
1,1649660085,1.0,"SMITH, KALEENA",235Z00000X,Speech-Language Pathologist,None,None,948 NECTAR CT,None,ADAMS,TN,37010,"948 NECTAR CT, ADAMS, TN 37010"
2,1811367980,1.0,"GRAMLICK, RENE",133V00000X,"Dietitian, Registered",None,None,2875 OLD CLARKSVILLE SPGFLD RD,None,ADAMS,TN,37010,"2875 OLD CLARKSVILLE SPGFLD RD, ADAMS, TN 37010"
3,1760929236,1.0,"HUFF, SYDNEY",390200000X,Student in an Organized Health Care Education/...,None,None,3835 HEAD RD,None,ADAMS,TN,37010,"3835 HEAD RD, ADAMS, TN 37010"
4,1740720275,2.0,None,261QM1300X,Clinic/Center,Multi-Specialty,CHS TENNESSEE MEDICAL PC,4538 GUTHRIE HWY,None,CLARKSVILLE,TN,37010,"4538 GUTHRIE HWY, CLARKSVILLE, TN 37010"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
44218,1194394767,1.0,"VADEN, CHARLI",225100000X,Physical Therapist,None,None,417 GORDONSVILLE HWY STE 100,None,GORDONSVILLE,TN,38563,"417 GORDONSVILLE HWY STE 100, GORDONSVILLE, TN..."
44219,1518611425,1.0,"SIRCY, ANNA",363LF0000X,Nurse Practitioner,Family,None,8 NEW MIDDLETON HWY,None,GORDONSVILLE,TN,38563,"8 NEW MIDDLETON HWY, GORDONSVILLE, TN 38563"
44220,1831561059,1.0,"MONTALBO, CYNTHIA",224Z00000X,Occupational Therapy Assistant,None,None,73 JUDKINS LN,None,HICKMAN,TN,38567,"73 JUDKINS LN, HICKMAN, TN 38567"
44221,1558875468,2.0,None,343900000X,Non-emergency Medical Transport (VAN),None,"BLUE HERON TRANSPORTATION, LLC",177 SPURLOCK LN,None,HICKMAN,TN,38567,"177 SPURLOCK LN, HICKMAN, TN 38567"


# Combine dataframes to make tn_full

In [186]:
pro_geo = profiles.merge(geo)

In [187]:
tncosts_geo = pro_geo.merge(tn_costs, on = 'npi')

In [188]:
tncosts_geo = tncosts_geo[tncosts_geo['entity_type_code'] == 1.0]

In [189]:
tncosts_geo_drops = ['entity_type_code', 
                     'name',
                     'taxonomy_code', 
                     'classification',
                     'specialization',
                     'organization', 
                     'address_line1', 
                     'address_line2']

tn_med_costs_geo = tncosts_geo.drop(columns = tncosts_geo_drops)

,npi,city,state,zip,address,lat,lng,last_name,first_name,credentials,...,procedure_code,procedure_description,place_of_service,total_beneficiaries,total_services,total_beneficiaries_per_day_services,avg_submitted_charge,avg_medicare_allowed,avg_medicare_payment,avg_medicare_standardized_payment
0,1043218654,ANTIOCH,TN,37013,"5801 CROSSINGS BLVD, ANTIOCH, TN 37013",36.037696,-86.642255,Miller,Timothy,M.D.,...,20552,Injections of trigger points in 1 or 2 muscles,O,15,18.0,18,231.176667,52.110000,41.630000,44.360000
1,1043218654,ANTIOCH,TN,37013,"5801 CROSSINGS BLVD, ANTIOCH, TN 37013",36.037696,-86.642255,Miller,Timothy,M.D.,...,20610,Aspiration and/or injection of large joint or ...,O,31,39.0,39,268.335385,58.047179,43.968974,48.745897
2,1043218654,ANTIOCH,TN,37013,"5801 CROSSINGS BLVD, ANTIOCH, TN 37013",36.037696,-86.642255,Miller,Timothy,M.D.,...,27096,Injection procedure into sacroiliac joint for ...,O,16,16.0,16,583.257500,150.920000,120.570000,128.840000
3,1043218654,ANTIOCH,TN,37013,"5801 CROSSINGS BLVD, ANTIOCH, TN 37013",36.037696,-86.642255,Miller,Timothy,M.D.,...,62321,Injection of substance into spinal canal of up...,F,15,18.0,18,917.498889,103.960556,78.417222,82.256667
4,1043218654,ANTIOCH,TN,37013,"5801 CROSSINGS BLVD, ANTIOCH, TN 37013",36.037696,-86.642255,Miller,Timothy,M.D.,...,62323,Injection of substance into spinal canal of lo...,F,65,103.0,103,874.569223,94.629709,73.686990,78.663398


In [190]:
tn_full = tn_med_costs_geo.merge(mips_thin, on = 'npi')
tn_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59098 entries, 0 to 59097
Data columns (total 26 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   npi                                   59098 non-null  int64  
 1   city                                  59098 non-null  object 
 2   state                                 59098 non-null  object 
 3   zip                                   59098 non-null  object 
 4   address                               59098 non-null  object 
 5   lat                                   59098 non-null  float64
 6   lng                                   59098 non-null  float64
 7   last_name                             59098 non-null  object 
 8   first_name                            59098 non-null  object 
 9   credentials                           57157 non-null  object 
 10  specialty                             59098 non-null  object 
 11  procedure_code 

In [191]:
tn_full.to_csv('data/danger_zone.csv', index = False)

# Explore tn_full

In [201]:
tn_full.groupby(['procedure_description', 'place_of_service'])['avg_submitted_charge'].agg(['mean', 'std']).reset_index()

,procedure_description,place_of_service,mean,std
0,3D radiographic procedure,F,33.257486,9.365621
1,3D radiographic procedure,O,172.344618,137.502783
2,3D radiographic procedure with computerized im...,F,110.352222,31.202294
3,3D radiographic procedure with computerized im...,O,110.568543,32.406121
4,5% dextrose/water (500 ml = 1 unit),O,20.000000,NaN
...,...,...,...,...
2022,"X-ray of wrist, 2 views",O,63.585373,17.356806
2023,"X-ray of wrist, minimum of 3 views",F,38.135544,13.672381
2024,"X-ray of wrist, minimum of 3 views",O,97.518674,22.845509
2025,X-ray survey of forearm or wrist bone density,F,116.923077,7.488453
